<a href="https://colab.research.google.com/github/IMTKU-for-NTCIR-15-Shared-Tasks/imtku_for_dial_eval_1/blob/develop/bert_token_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
"""
參考文章:https://leemeng.tw/attack_on_bert_transfer_learning_in_nlp.html
"""

'\n參考文章:https://leemeng.tw/attack_on_bert_transfer_learning_in_nlp.html\n'

In [0]:
%%bash
pip install transformers tqdm boto3 requests regex -q

In [0]:
import torch
from transformers import BertModel
from transformers import BertTokenizer

PRETRAINED_MODEL_NAME = "hfl/chinese-bert-wwm" # 指定繁簡中文 BERT-wwm (全詞遮罩) 預訓練模型

model = BertModel.from_pretrained(PRETRAINED_MODEL_NAME)
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL_NAME) # 取得此預訓練模型所使用的 tokenizer

print("PyTorch 版本:",torch.__version__)


PyTorch 版本: 1.5.0+cu101


In [0]:
vocab = tokenizer.vocab
print("字典大小:",len(vocab))
print(vocab['[CLS]'])
print(vocab['[SEP]'])

字典大小: 21128
101
102


In [0]:
import random
random_tokens = random.sample(list(vocab),10) # sample 10個
random_ids = [vocab[t] for t in random_tokens]  #index如何產生??
print("{0:20}{1:15}".format("token","index"))
print("-" * 25)
for t,id in zip(random_tokens,random_ids): #zip:將兩個list包裝成一組
    print("{0:15}{1:10}".format(t,id))


token               index          
-------------------------
##09                 9267
╮                     444
water               11060
##cca               13144
沉                    3756
歎                    3626
缩                    5367
##吶                 14485
はお                  13232
1884                13007


In [0]:
text = "[CLS]等到潮水退了，就知道誰沒穿褲子。"
tokens = tokenizer.tokenize(text)
ids = tokenizer.convert_tokens_to_ids(tokens)

print(text)
print(tokens)
print(ids)

[CLS]等到潮水退了，就知道誰沒穿褲子。
['[CLS]', '等', '到', '潮', '水', '退', '了', '，', '就', '知', '道', '誰', '沒', '穿', '褲', '子', '。']
[101, 5023, 1168, 4060, 3717, 6842, 749, 8024, 2218, 4761, 6887, 6306, 3760, 4959, 6194, 2094, 511]


In [0]:
from google.colab import drive
import pandas as pd

drive.mount('/content/gdrive') # 此處需要登入google帳號
# 獲取授權碼之後輸入即可連動雲端硬碟
%cd '/content/gdrive/My Drive/Colab Notebooks'

# df_train = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/train.csv")
# 取得Dataframe
df_train = pd.read_csv('train.csv') # 訓練資料
# Dataset is now stored in a Pandas Dataframe

print("訓練樣本數：", len(df_train))
df_train.head(20)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks
訓練樣本數： 673


,text,intent
0,什麼航班資訊確實長榮航空從出發到西雅圖 名古屋,flight_time
1,什麼是西雅圖到名古屋的最低價的票價,airfare
2,讓我從西雅圖所有最低價的票價名古屋留下星期三,airfare
3,什麼是西雅圖的最低價的票價名古屋任何類,airfare
4,需要多少費用從西雅圖飛轉機到名古屋上長榮航空,airfare
5,什麼才是從西雅圖到名古屋的長榮航空次航班票價,airfare
6,什麼是西雅圖和名古屋之間的最低價的機票,airfare
7,給我一月 四點的航班和票價從西雅圖到名古屋,flight+airfare
8,什麼航空公司有從西雅圖到名古屋 轉機 早上的 早上的航班,airline
9,該航空公司從西雅圖到名古屋飛,airline


In [0]:
df_train.intent.value_counts() / len(df_train) # 看訓練集每個intent的比例

airfare                      0.569094
airline                      0.167905
flight_time                  0.065379
aircraft                     0.052006
flight                       0.034175
flight+airfare               0.029718
quantity                     0.022288
city                         0.017831
flight_no                    0.014859
airport                      0.013373
meal                         0.004458
airline+flight_no            0.002972
airfare+flight_time          0.001486
cheapest                     0.001486
distance                     0.001486
aircraft+flight+flight_no    0.001486
Name: intent, dtype: float64

In [0]:
"""
# idempotence, 將處理結果另存成 tsv 供 PyTorch 使用
df_train.to_csv("train.tsv", sep="\t", index=False)  #

from google.colab import files
files.download("train.tsv")
"""

'\n# idempotence, 將處理結果另存成 tsv 供 PyTorch 使用\ndf_train.to_csv("train.tsv", sep="\t", index=False)  #\n\nfrom google.colab import files\nfiles.download("train.tsv")\n'

In [0]:
""" 
此 Dataset 每次將 tsv 裡的一筆成對句子轉換成 BERT 相容的格式，並回傳 3 個 tensors：
- tokens_tensor：兩個句子合併後的索引序列，包含 [CLS] 與 [SEP]
- segments_tensor：可以用來識別兩個句子界限的 binary tensor 
- label_tensor：將分類標籤轉換成類別索引的 tensor, 如果是測試集則回傳 None
"""
"""
如果輸入只有一個句子就將segments_tensor全部設為0。
!!錯誤:segments_tensor = 0 。
換成segments_tensor = torch.tensor([0] * length, dtype=torch.long) 就OK了 (edited)

參考:https://www.cnblogs.com/d0main/p/10447853.html#%E5%AE%9E%E7%8E%B0
"""
from torch.utils.data import Dataset

class ATIS_zh_Dataset(Dataset):
    # 讀取前處理後的 tsv 檔並初始化一些參數
    def __init__(self, mode ,tokenizer):
        assert mode in ["train" , "test"] # 如果mode不等於這兩個，assert Exception
        self.mode = mode
        self.df = pd.read_csv(mode + ".tsv" , sep="\t").fillna("") # 把Na(空值)取代
        self.len = len(self.df)
        self.intent_map = {"aircraft":0,"aircraft+flight+flight_no":1,"airfare":2,"airfare+flight_time":3,"airline":4,"airline+flight_no":5,"airport":6,
                           "cheapest":7,"city":8,"distance":9,"flight":10,"flight_no":11,"flight_time":12,"flight+airfare":13,"meal":14,"quantity":15}
        self.tokenizer = tokenizer

     # 定義回傳一筆訓練 / 測試數據的函式
    def __getitem__(self , index):
            
        text, intent = self.df.iloc[index, :].values
        # 將 label 文字也轉換成索引方便轉換成 tensor
        intent_id = self.intent_map[intent]
        intent_tensor = torch.tensor(intent_id)
        
        
        
        word_pieces = ["[CLS]"]
        tokens = self.tokenizer.tokenize(text)
        word_pieces += tokens 
        length = len(word_pieces)

        # 將第一句包含 [SEP] 的 token 位置設為 0，其他為 1 表示第二句
        segments_tensor = torch.tensor([0] * length, 
                                        dtype=torch.long)
        
        # 將整個 token 序列轉換成索引序列
        indexs = self.tokenizer.convert_tokens_to_ids(word_pieces)
        tokens_tensor = torch.tensor(indexs)

        return(tokens_tensor,segments_tensor,intent_tensor)
      
    def __len__(self):
        return self.len
        
# 初始化一個專門讀取訓練樣本的 Dataset，使用中文 BERT 斷詞
trainset = ATIS_zh_Dataset("train", tokenizer=tokenizer)


# print(trainset[0])
# print(trainset[0][2])




In [0]:
# 選擇第一個樣本
sample_idx = 0

# 將原始文本拿出做比較
text, intent = trainset.df.iloc[sample_idx].values

# 利用剛剛建立的 Dataset 取出轉換後的 id tensors
tokens_tensor, segments_tensor, intent_tensor = trainset[sample_idx]

# 將 tokens_tensor 還原成文本
tokens = tokenizer.convert_ids_to_tokens(tokens_tensor.tolist())
combined_text = "".join(tokens)

# 渲染前後差異
print(f"""[原始文本]
句子 1：{text}
分類  ：{intent}

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：{tokens_tensor}

segments_tensor：{segments_tensor}

label_tensor   ：{intent_tensor}

--------------------

[還原 tokens_tensors]
{combined_text}
""")

[原始文本]
句子 1：什麼航班資訊確實長榮航空從出發到西雅圖 名古屋
分類  ：flight_time

--------------------

[Dataset 回傳的 tensors]
tokens_tensor  ：tensor([ 101,  784, 7938, 5661, 4408, 6536, 6244, 4825, 2179, 7269, 3532, 5661,
        4958, 2537, 1139, 4634, 1168, 6205, 7414, 1756, 1399, 1367, 2238])

segments_tensor：tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

label_tensor   ：12

--------------------

[還原 tokens_tensors]
[CLS]什麼航班資訊確實長榮航空從出發到西雅圖名古屋



In [0]:
"""
實作可以一次回傳一個 mini-batch 的 DataLoader
這個 DataLoader 吃我們上面定義的 `ATIS_zh_Dataset`，
回傳訓練 BERT 時會需要的 4 個 tensors：
- tokens_tensors  : (batch_size, max_seq_len_in_batch)
- segments_tensors: (batch_size, max_seq_len_in_batch)
- masks_tensors   : (batch_size, max_seq_len_in_batch)
- label_ids       : (batch_size)
"""

from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

# 這個函式的輸入 `samples` 是一個 list，裡頭的每個 element 都是
# 剛剛定義的 `ATIS_zh_Dataset` 回傳的一個樣本，每個樣本都包含 3 tensors：
# - tokens_tensor
# - segments_tensor
# - label_tensor
# 它會對前兩個 tensors 作 zero padding，並產生前面說明過的 masks_tensors

def create_mini_batch(samples):
    tokens_tensors = [s[0] for s in samples]
    segments_tensors = [s[1] for s in samples]
    
    # 測試集有 labels

    label_ids = torch.stack([s[2] for s in samples])
    
    # zero pad 到同一序列長度
    tokens_tensors = pad_sequence(tokens_tensors, 
                                  batch_first=True)
    segments_tensors = pad_sequence(segments_tensors, 
                                    batch_first=True)
    
    # attention masks，將 tokens_tensors 裡頭不為 zero padding
    # 的位置設為 1 讓 BERT 只關注這些位置的 tokens
    masks_tensors = torch.zeros(tokens_tensors.shape, 
                                dtype=torch.long)
    masks_tensors = masks_tensors.masked_fill(
        tokens_tensors != 0, 1)
    
    return tokens_tensors, segments_tensors, masks_tensors, label_ids

# 初始化一個每次回傳 64 個訓練樣本的 DataLoader
# 利用 `collate_fn` 將 list of samples 合併成一個 mini-batch 是關鍵
BATCH_SIZE = 64
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, 
                         collate_fn=create_mini_batch)

In [0]:
data = next(iter(trainloader)) 

tokens_tensors, segments_tensors, masks_tensors, label_ids = data

print(f"""
tokens_tensors.shape   = {tokens_tensors.shape} 
{tokens_tensors}
------------------------
segments_tensors.shape = {segments_tensors.shape}
{segments_tensors}
------------------------
masks_tensors.shape    = {masks_tensors.shape}
{masks_tensors}
------------------------
label_ids.shape        = {label_ids.shape}
{label_ids}
""")


tokens_tensors.shape   = torch.Size([64, 38]) 
tensor([[ 101,  784, 7938,  ...,    0,    0,    0],
        [ 101,  784, 7938,  ...,    0,    0,    0],
        [ 101, 6366, 2769,  ...,    0,    0,    0],
        ...,
        [ 101, 6205, 7414,  ...,    0,    0,    0],
        [ 101,  784, 7938,  ...,    0,    0,    0],
        [ 101,  784, 7938,  ...,    0,    0,    0]])
------------------------
segments_tensors.shape = torch.Size([64, 38])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
------------------------
masks_tensors.shape    = torch.Size([64, 38])
tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])
------------------------
label_ids.shape        

In [0]:
from transformers import BertForTokenClassification

PRETRAINED_MODEL_NAME = "hfl/chinese-bert-wwm"
NUM_LABELS = 6  

model = BertForTokenClassification.from_pretrained(PRETRAINED_MODEL_NAME,
                                                      num_labels=NUM_LABELS)

print("""
name            module
----------------------""")
for name, module in model.named_children():
    if name == "bert":
        for n, _ in module.named_children():
            print(f"{name}:{n}")
    else:
        print("{:15} {}".format(name, module))


name            module
----------------------
bert:embeddings
bert:encoder
bert:pooler
dropout         Dropout(p=0.1, inplace=False)
classifier      Linear(in_features=768, out_features=6, bias=True)


In [0]:
""" (以下程式碼未更改)
定義一個可以針對特定 DataLoader 取得模型預測結果以及分類準確度的函式
之後也可以用來生成上傳到 Kaggle 競賽的預測結果

2019/11/22 更新：在將 `tokens`、`segments_tensors` 等 tensors
丟入模型時，強力建議指定每個 tensor 對應的參數名稱，以避免 HuggingFace
更新 repo 程式碼並改變參數順序時影響到我們的結果。
"""

def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
      
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            
            # 別忘記前 3 個 tensors 分別為 tokens, segments 以及 masks
            # 且強烈建議在將這些 tensors 丟入 `model` 時指定對應的參數名稱
            tokens_tensors, segments_tensors, masks_tensors = data[:3]
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors)
            
            logits = outputs[0]
            _, pred = torch.max(logits.data, -1)
            
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions
    
# 讓模型跑在 GPU 上並取得訓練集的分類準確率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
_, acc = get_predictions(model, trainloader, compute_acc=True)
print("classification acc:", acc)

device: cuda:0


RuntimeError: ignored

In [0]:
"""(以下程式碼未更改，非重點)"""
def get_learnable_params(module):
    return [p for p in module.parameters() if p.requires_grad]
     
model_params = get_learnable_params(model)
clf_params = get_learnable_params(model.classifier)

print(f"""
整個分類模型的參數量：{sum(p.numel() for p in model_params)}
線性分類器的參數量：{sum(p.numel() for p in clf_params)}
""")

In [0]:
%%time

# 訓練模式
model.train()

# 使用 Adam Optim 更新整個分類模型的參數
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)


EPOCHS = 6  # 訓練次數  -----------------------------------------------------------------------------------
for epoch in range(EPOCHS):
    
    running_loss = 0.0
    for data in trainloader:
        
        tokens_tensors, segments_tensors, \
        masks_tensors, labels = [t.to(device) for t in data]

        # 將參數梯度歸零
        optimizer.zero_grad()
        
        # forward pass
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)

        loss = outputs[0]
        # backward
        loss.backward()
        optimizer.step()


        # 紀錄當前 batch loss
        running_loss += loss.item()
        
    # 計算分類準確率
    _, acc = get_predictions(model, trainloader, compute_acc=True)

    print('[epoch %d] loss: %.3f, acc: %.3f' %
          (epoch + 1, running_loss, acc))
    

In [0]:
%%time 
# 關注這個準確率!!!(越高越好)
# 建立測試集。這邊我們可以用跟訓練時不同的 batch_size，看你 GPU 多大
testset = ATIS_zh_Dataset("train" , tokenizer=tokenizer)

testloader = DataLoader(testset, batch_size=16, 
                        collate_fn=create_mini_batch)

# 用分類模型預測測試集
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
_, acc = get_predictions(model, testloader, compute_acc=True)
print("TestDataset classification acc:", acc)



In [0]:
input_sentence ="我想從台灣去日本"
print('tokens_num:',len(input_sentence))
inputs = tokenizer.encode_plus(text=input_sentence,return_tensors='pt',add_special_tokens=True)
print(inputs)

input_ids = inputs['input_ids']
token_type_ids = inputs['token_type_ids']
attention_mask = inputs['attention_mask']
print("--------------------------------------------")
input_ids = input_ids.to(device)
token_type_ids = token_type_ids.to(device)
attention_mask = attention_mask.to(device)
model = model.to(device)

outputs = model(input_ids=input_ids,
                token_type_ids=token_type_ids,
                attention_mask=attention_mask,labels=None)
#print('outputs:',outputs[0]) 
#print(outputs[0]) #18個logits
#print(outputs[0][0]) #每個token的logits outputs[0][0]~outputs[0][17]
all_logits = outputs[0]
print(all_logits.tolist()) 
_,pred = torch.max(all_logits,2) #???? 1. torch.max(all_logits,1)抓每一"行"的最大值  2. torch.max(all_logits,2)抓每一"列"的最大值 
print(pred)
print("--------------------------------------------")
token_class_dic = {0:" O ",1:"[fromloc.city_name]",2:"[airline_name]",3:"[depart_time.time]",4:"[class_type]",5:"[toloc.city_name]"}
print('text:',input_sentence)
tokens_num = len(pred[0]); #18個包含[CLS][SEP]
org=""
for i in range(1,tokens_num-1):
    org+=token_class_dic[pred[0][i].item()]
print(org)
print('tokens_class_num:',tokens_num)

tokens_num: 8
{'input_ids': tensor([[ 101, 2769, 2682, 2537, 1378, 4124, 1343, 3189, 3315,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
--------------------------------------------
[[[-0.04524211958050728, 0.2431921362876892, 0.28879985213279724, -0.6827070713043213, 0.012832682579755783, -0.24158097803592682], [0.3243280053138733, 0.11394919455051422, -0.6831933856010437, -0.32468828558921814, -0.00454736826941371, -0.13152159750461578], [0.46534305810928345, 0.5242226719856262, -0.8275352120399475, -0.5407520532608032, -0.056216761469841, -0.0975010022521019], [0.5279955863952637, 0.6008447408676147, -0.4545857012271881, -0.456890732049942, -0.36250609159469604, -0.15283454954624176], [0.1631951630115509, 0.6476370096206665, -0.5040625333786011, -0.7852661609649658, 0.03768455237150192, -0.07913783192634583], [0.21633096039295197, 0.5018258094787598, -0.6232933402061462, -0.5055908560752869, 0.2405267

In [0]:
from transformers import BertTokenizer, BertForTokenClassification
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained('bert-base-uncased')

input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
labels = torch.tensor([1] * input_ids.size(1)).unsqueeze(0)  # Batch size 1
print(labels)
outputs = model(input_ids, labels=labels)
#print(outputs)
loss, scores = outputs[:2]
print(scores)

In [0]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
print(labels)
outputs = model(input_ids, labels=labels)

loss, logits = outputs[:2]